# Przetwarzanie Grafiki i Muzyki - laboratorium nr 9: Operacje „Hit-and-Miss” („Hit-or-Miss”)


  * Bazowa operacja morfologii matematycznej.
  * W notebooku będziemy pracować na elementach strukturalnych, które mają następujące elementy:
    * Piksele obiektu ('1')
    * Piksele tła ('0')
    * Piksele nieokreślone – niebrane pod uwagę ('x')

  * Obraz wejściowy jest przeglądany piksel po pikselu, a następnie, jeśli konfiguracja pikseli sąsiednich jest identyczna jak w elemencie strukturalnym (ang. structural element, w skrócie SE), to jest on ustawiany na wartość '1', w przeciwnym przypadku na '0'.
  * Aby wykryć daną cechę, wyniki kilku SE mogą się sumować/mnożyć.
  
  
## Hit-or-miss - w skrócie

  * Przykładamy środek obiektu strukturalnego do każdego piksela obrazka i jeśli pasuje on do sąsiedztwa danego piksela, to zamieniam piksel na biały, a jeśli nie to na czarny.
  * Przydatny link z wyjaśnieniem: https://theailearner.com/tag/hit-or-miss-transformation-opencv/

## Zadanie 1 (2 punkty)
  * Proszę dokonac binaryzacji obrazu LENA_512 algorytmem Otsu.
  * Proszę wykonać operację Hit-or-miss przy pomocy danego elementu strukturalnego:


$$
\begin{bmatrix} 
1 & 1 & 1 \\
1 & x & 1 \\
1 & 1 & 1 \\
\end{bmatrix} 
$$

Hint: Aby wykonać to zadanie, proszę spojrzeć na dokumentację metody morphologyEx(...) z pakietu OpenCV.

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# myslcie o x jako o -1
def binarize_otsu(image):
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    flag = cv2.THRESH_BINARY + cv2.THRESH_OTSU
    ret, binarized_image = cv2.threshold(ycbcr_image[:,:,0], 0, 255, flag)
    return binarized_image


def hit_or_miss(image):
    image_binarized = binarize_otsu(image)
    kernel = np.array([[1, 1, 1],
                       [1, -1, 1],
                       [1, 1, 1]], dtype=np.uint8)
    output_image = cv2.morphologyEx(image_binarized, cv2.MORPH_HITMISS, kernel)
    return output_image


lena = cv2.imread("./img/LENA_512.jpg", cv2.IMREAD_COLOR)
hit_or_miss_lena = hit_or_miss(lena)
cv2.imshow("lena", lena)
cv2.imshow("hit_or_miss_lena", hit_or_miss_lena)


cv2.waitKey(50)
cv2.destroyAllWindows()

/var/folders/6t/lw9p33tn1q124mgvz_z7v9gm0000gn/T/ipykernel_3734/1015737056.py:10: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -1 to uint8 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  kernel = np.array([[1, 1, 1],


:## Operacja pogrubiania – wypukły kształt (convex hull)

Naszym celem jest znalezienie wypukłej otoczki obiektów.

  * Używanych jest 8 SE:
    * 2 wzorcowe:

$$
\begin{bmatrix} 
1 & 1 & x \\
1 & 0 & x \\
1 & x & 0 \\
\end{bmatrix} 
$$

oraz

$$
\begin{bmatrix} 
x & 1 & 1 \\
x & 0 & 1 \\
0 & x & 1
\end{bmatrix} 
$$
  * Po 3 SE z każdego wzorcowego obrócone o 90, 180, 270 stopni.

  * Procedura iteracyjna: po osiągnięciu zbieżności kolejne iteracje nie zmieniają wyniku (procedura zatrzymuje się sama)

## Zadanie 2 (4 punkty)

Proszę zbudować otoczkę wypukłą elementów na https://raw.githubusercontent.com/przem85/PGiM/master/img/convex.png. Proszę pokazać efekty skończonej procedury.

HINT: Aby zbudować otoczkę wypukłą na obiekcie, musimy najpierw znaleźć jego kształt!

In [13]:
# cv2.convexHull
# 8 SE tworzy sie automatycznie
# w opencv
# - wczytac obrazek
# - binaryzacja
# - findContours
# - convexhull

def binarize_otsu(image):
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    flag = cv2.THRESH_BINARY + cv2.THRESH_OTSU
    ret, binarized_image = cv2.threshold(ycbcr_image[:,:,0], 0, 255, flag)
    return binarized_image


def convex_hull(image):
    output_image = np.copy(image)
    image_binarized = binarize_otsu(image)
    contours, _ = cv2.findContours(image_binarized, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
    cv2.drawContours(output_image, contours, -1, (0,255,0), 3)    
    return output_image


convex = cv2.imread("./img/convex.png", cv2.IMREAD_COLOR)
convex_contours = convex_hull(convex)
cv2.imshow("convex", convex)
cv2.imshow("convex_contours", convex_contours)


cv2.waitKey(50)
cv2.destroyAllWindows()

# Operacje pocieniania – znajdowanie szkieletu obiektu

  * Naszym celem jest znalezienie szkieletu obiektu – linii reprezentujących obiekt.

  * iteracyjne zmniejszanie obiektów – 8 SE powstałych z obrotów 2 SE:

$$
\begin{bmatrix} 
1 & 1 & x \\
1 & 0 & x \\
1 & x & 0 \\
\end{bmatrix} 
$$

oraz

$$
\begin{bmatrix} 
x & 1 & 1 \\
x & 0 & 1 \\
0 & x & 1
\end{bmatrix} 
$$

# Zadanie 3 (4 punkty)
Proszę zbudować szkielet powyższą metodą na zbinaryzowanych (algorytmem 0tsu) obrazkach (Uwaga: Obrazki poniżej nie są binarne). Proszę wykonać zadanie na dwóch wybranych obrazkach:

  * https://raw.githubusercontent.com/przem85/PGiM/master/img/template1.png.
  * https://raw.githubusercontent.com/przem85/PGiM/master/img/template2.png.
  * https://raw.githubusercontent.com/przem85/PGiM/master/img/template3.png.
  * https://raw.githubusercontent.com/przem85/PGiM/master/img/template4.png.
  * https://raw.githubusercontent.com/przem85/PGiM/master/img/template5.png.
  * https://raw.githubusercontent.com/przem85/PGiM/master/img/template6.png.

Proszę pokazać efekty z kilku iteracji.

HINT: Proszę rzucić okiem na ten blog http://opencvpython.blogspot.com/2012/05/skeletonization-using-opencv-python.html

In [18]:
def skeleton(image):
    _, image = cv2.threshold(image, 127, 255, 0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    size = np.size(image)
    skel = np.zeros(image.shape, dtype=np.uint8)
    done = False
    while not done:
        eroded = cv2.erode(image, element)
        temp = cv2.dilate(eroded, element)
        temp = cv2.subtract(image, temp)
        skel = cv2.bitwise_or(skel, temp)
        image = eroded.copy()
    
        zeros = size - cv2.countNonZero(image)
        if zeros == size:
            done = True
    
    return skel

template1 = cv2.imread("./img/template1.png", 0)
template2 = cv2.imread("./img/template2.png", 0)
cv2.imshow("template1", template1)
cv2.imshow("template2", template2)

skeleton_template1 = skeleton(template1)
skeleton_template2 = skeleton(template2)
cv2.imshow("skeleton_template1", skeleton_template1)
cv2.imshow("skeleton_template2", skeleton_template2)

cv2.waitKey(50)
cv2.destroyAllWindows()